In [1]:
!pip install openai
!pip install langchain_openai
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 640.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.4/337.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.5/127.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 4.7 MB/s eta 0:00:00


In [10]:
from openai import OpenAI

client = OpenAI(api_key="sk-")

# Define the questionnaire answers
answers = {
    "company_a": "Sodexo",
    "industry": "facilities management and food services",
    "company_b": "CoachHub",
    "goal": "accelerate their leadership development across APMEA",
    "achievement_1": "Scalable coaching programs powered by AI and analytics with local support",
    "achievement_2": "Access to a diverse pool of 3,500 top certified coaches from around the world",
    "achievement_3": "Unlimited, structured coaching sessions and flexibility 24/7 for a hybrid, mobile workforce",
    "achievement_4": "Boost knowledge retention and engagement with global benchmarking",
    "achievement_5": "Quick implementation with low administrative burden",
    "professionals": "coachees",
    "individual": "Sodexo's Head of FMCG Accounts, Jean Baptiste CALEMARD",
    "cta": "explore the greater you",
    "link": "https://bit.ly/3LMxIHg"
}

# Define the prompt template with placeholders
system_message = """
You are tasked with writing a social media post that highlights a successful collaboration between two entities. The tone should be professional, and the achievements should be presented in bullet points. Here's the format and style you should follow:
𝗗𝗶𝗱 𝘆𝗼𝘂 𝗸𝗻𝗼𝘄? 75% of classroom-style training is forgotten if it's not implemented within 6 days after.

Discover how {company_a}, a leader in {industry}, partnered with {company_b} to achieve {goal}:

🚀 {achievement_1}
🌎 {achievement_2}
🌟 {achievement_3}
🧠 {achievement_4}
✅ {achievement_5}

Join thousands of {professionals}, like {individual}, on a journey of growth and transformation to {cta}:
{link}
"""

# Fill in the placeholders with the actual answers
filled_system_message = system_message.format(
    company_a=answers["company_a"],
    industry=answers["industry"],
    company_b=answers["company_b"],
    goal=answers["goal"],
    achievement_1=answers["achievement_1"],
    achievement_2=answers["achievement_2"],
    achievement_3=answers["achievement_3"],
    achievement_4=answers["achievement_4"],
    achievement_5=answers["achievement_5"],
    professionals=answers["professionals"],
    individual=answers["individual"],
    cta=answers["cta"],
    link=answers["link"]
)

# Make LLM call for response
response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {
            "role": "system",
            "content": filled_system_message
        }
    ],
    temperature=0.8,
    top_p=1
)

# Extract and print the generated post
post = response.choices[0].message.content.strip()
print("Generated Post:\n", post)

Generated Post:
 Did you know? A powerful collaboration can be a game-changer in achieving accelerated leadership development. Let's delve into the successful alliance between Sodexo and CoachHub that has brought about a significant transformation in the APMEA region. 

✅ Sodexo, a global player in facilities management and food services, joined hands with CoachHub, a frontrunner in digital coaching platforms. 

Here's a look at the breakthrough they achieved together:

🔹 Leveraged AI and analytics to roll out scalable coaching programs, backed by local support
🔹 Provided access to a diverse pool of 3,500 top certified coaches from across the globe
🔹 Created a structure for unlimited, round-the-clock coaching sessions to cater to a hybrid, mobile workforce
🔹 Enhanced knowledge retention and engagement through global benchmarking
🔹 Ensured speedy implementation with minimal administrative workload

The partnership has helped many, including Jean Baptiste CALEMARD, Sodexo's Head of FMCG 

In [12]:
# Store the conversation history
conversation_history = [
    {"role": "system", "content": "You are a social media manager generating posts based on given data."},
    {"role": "user", "content": filled_system_message},
    {"role": "assistant", "content": post}
]

# Define a function to ask questions about the post
def ask_question_about_post(question):
    # Append the user's question to the conversation history
    conversation_history.append({"role": "user", "content": question})

    # Define the prompt for answering questions
    prompt = """
    You are an assistant answering questions about a generated social media post. Please provide concise, to the point and relevant responses based on the provided post. Make sure to include specific details and examples from the post where applicable.

    Post:
    {post}

    Question: {question}
    Answer:
"""

    # Fill in the placeholders with the actual post and question
    filled_prompt = prompt.format(post=post, question=question)

    # Make the API call with the updated conversation history
    interaction_response = client.chat.completions.create(
        model="gpt-4",
        messages=conversation_history + [{"role": "user", "content": filled_prompt}],
        temperature=0.8,
        top_p=1
    )

    # Extract and print the response
    answer = interaction_response.choices[0].message.content.strip()

    # Append the assistant's response to the conversation history
    conversation_history.append({"role": "assistant", "content": answer})

    return answer

# Example questions
questions = [
    "Can you describe the specific case study or transformation story you want to share?",
    "What challenge or problem was addressed in this case?",
    "What were the key results or outcomes achieved in this story?",
    "Are there any data, quotes, or testimonials that illustrate the impact?",
    "Is there a specific call-to-action?",
    "Are there any specific hashtags you’d like to include?",
    "Describe your desired tone and style."
]

# Ask questions and print the responses
for question in questions:
    answer = ask_question_about_post(question)
    print(f"Q: {question}\nA: {answer}\n")

Q: Can you describe the specific case study or transformation story you want to share?
A: The specific transformation story focuses on the successful collaboration between Sodexo and CoachHub, two entities that are leaders in their respective sectors. Together, they have revolutionized leadership development in the APMEA region.

The partnership enabled Sodexo to leverage CoachHub's AI and analytics-powered platform to roll out scalable coaching programs. These programs have a diverse pool of over 3,500 top certified coaches worldwide, enabling Sodexo to create a structure for unlimited, 24/7 coaching sessions catering to a hybrid, mobile workforce.

This collaboration resulted in enhanced knowledge retention and engagement through global benchmarking, achieved with minimal administrative workload. A significant transformation story is of Jean Baptiste CALEMARD, Sodexo's Head of FMCG Accounts, who embarked on a journey of growth and transformation through this partnership.

To explore 